<a href="https://colab.research.google.com/github/Fionachai/Recommendation-system/blob/main/Hybrid_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

In [ ]:
import pandas as pd

# Load data from CSV
leet = pd.read_csv('https://raw.githubusercontent.com/Fionachai/csvfiles/main/leet.csv')
data = pd.read_csv('https://raw.githubusercontent.com/Fionachai/csvfiles/main/random_data.csv')

# User-Item Interactions Data
user_item_data = data[['User_ID','Item_id', 'Time','No_attempts']]

# Item Metadata
item_metadata = leet[['Item_id', 'title', 'difficulty', 'related_topics', 'similar_questions']]

# Save separated data to CSV files
user_item_data.to_csv('user_item_data.csv', index=False)
item_metadata.to_csv('item_metadata.csv', index=False)


In [ ]:
user_item_data.head()

,User_ID,Item_id,Time,No_attempts
0,39,1,18,3
1,41,2,8,3
2,1,3,7,2
3,75,4,6,2
4,48,5,9,5


In [ ]:
!pip install scikit-surprise

In [ ]:
# Load data from CSV
users_items_df = pd.read_csv('user_item_data.csv')
items_df = pd.read_csv('item_metadata.csv')



In [ ]:
# Collaborative Filtering
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(users_items_df[['User_ID', 'Item_id', 'No_attempts']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
cf_model = SVD()
cf_model.fit(trainset)

In [ ]:
# Content-Based Filtering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
item_tfidf_matrix = tfidf_vectorizer.fit_transform(items_df['title'])
content_sim_matrix = cosine_similarity(item_tfidf_matrix, item_tfidf_matrix)

In [ ]:
# Hybrid Recommendation
def hybrid_recommendation(User_ID, Item_id):
    # Collaborative Filtering
    cf_prediction = cf_model.predict(User_ID, Item_id).est

    # Content-Based Filtering
    item_idx = items_df[items_df['Item_id'] == Item_id].index[0]
    content_scores = list(enumerate(content_sim_matrix[item_idx]))
    content_scores = sorted(content_scores, key=lambda x: x[1], reverse=True)
    content_scores = content_scores[1:6]  # Exclude the item itself
    content_indices = [i[0] for i in content_scores]

    # Calculate hybrid score (simple average)
    hybrid_scores = [(cf_prediction + content_sim_matrix[item_idx, i]) / 2 for i in content_indices]

    # Get top recommendations
    top_indices = sorted(range(len(hybrid_scores)), key=lambda i: hybrid_scores[i], reverse=True)[:5]
    top_items = items_df.iloc[content_indices].iloc[top_indices]
    return top_items

In [ ]:
# Example usage
User_ID = 1
Item_id = 101
recommendations = hybrid_recommendation(User_ID, Item_id)
print(recommendations)


      Item_id                                   title difficulty  \
99        100                               Same Tree       Easy   
1515     1516             Move Sub-Tree of N-Ary Tree       Hard   
1037     1038  Binary Search Tree to Greater Sum Tree     Medium   
430       431        Encode N-ary Tree to Binary Tree       Hard   
653       654                     Maximum Binary Tree     Medium   

                                         related_topics  \
99                              Tree,Depth-first Search   
1515                                               Tree   
1037  Tree,Depth-first Search,Binary Search Tree,Rec...   
430                                                Tree   
653                                                Tree   

                                      similar_questions  
99                                                  NaN  
1515  [Find Root of N-Ary Tree, /problems/find-root-...  
1037                                                NaN  
430 